---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

<IPython.core.display.Javascript object>

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {
    "OH": "Ohio",
    "KY": "Kentucky",
    "AS": "American Samoa",
    "NV": "Nevada",
    "WY": "Wyoming",
    "NA": "National",
    "AL": "Alabama",
    "MD": "Maryland",
    "AK": "Alaska",
    "UT": "Utah",
    "OR": "Oregon",
    "MT": "Montana",
    "IL": "Illinois",
    "TN": "Tennessee",
    "DC": "District of Columbia",
    "VT": "Vermont",
    "ID": "Idaho",
    "AR": "Arkansas",
    "ME": "Maine",
    "WA": "Washington",
    "HI": "Hawaii",
    "WI": "Wisconsin",
    "MI": "Michigan",
    "IN": "Indiana",
    "NJ": "New Jersey",
    "AZ": "Arizona",
    "GU": "Guam",
    "MS": "Mississippi",
    "PR": "Puerto Rico",
    "NC": "North Carolina",
    "TX": "Texas",
    "SD": "South Dakota",
    "MP": "Northern Mariana Islands",
    "IA": "Iowa",
    "MO": "Missouri",
    "CT": "Connecticut",
    "WV": "West Virginia",
    "SC": "South Carolina",
    "LA": "Louisiana",
    "KS": "Kansas",
    "NY": "New York",
    "NE": "Nebraska",
    "OK": "Oklahoma",
    "FL": "Florida",
    "CA": "California",
    "CO": "Colorado",
    "PA": "Pennsylvania",
    "DE": "Delaware",
    "NM": "New Mexico",
    "RI": "Rhode Island",
    "MN": "Minnesota",
    "VI": "Virgin Islands",
    "NH": "New Hampshire",
    "MA": "Massachusetts",
    "GA": "Georgia",
    "ND": "North Dakota",
    "VA": "Virginia",
}

<IPython.core.display.Javascript object>

In [4]:
def get_list_of_university_towns():
    """Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'."""

    university_towns_df = pd.read_csv(
        "university_towns.txt",
        engine="python",
        sep="\n",
        header=None,
        names=["RegionName"],
    )
    university_towns_df["RegionName"] = university_towns_df["RegionName"].str.replace(
        r" ?\[[0-9]*\]", ""
    )
    university_towns_df["State"] = university_towns_df["RegionName"].apply(
        lambda x: re.sub(r"\[edit\]", "", x)
        if re.search(r"\[edit\]", x) != None
        else np.nan
    )

    university_towns_df["State"].fillna(method="ffill", inplace=True)

    university_towns_df["University"] = university_towns_df["RegionName"].apply(
        lambda x: re.sub(r"[\(\)]", "", re.search(r" ?\(.*\)?", x)[0])
        if re.search(r" ?\(.*\)?", x) != None
        else np.nan
    )
    university_towns_df["RegionName"] = university_towns_df["RegionName"].str.replace(
        r" ?\(.*\)?", ""
    )

    university_towns_df = university_towns_df[
        ~(
            university_towns_df["RegionName"].str.contains(r"\[edit\]")
            & university_towns_df["University"].isnull()
        )
    ]

    university_towns_df.reset_index(drop=True, inplace=True)

    return university_towns_df[["State", "RegionName"]]

<IPython.core.display.Javascript object>

In [5]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


<IPython.core.display.Javascript object>

In [6]:
def get_recession_start():
    """Returns the year and quarter of the recession start time as a
    string value in a format such as 2005q3"""
    gdp_df = pd.read_excel("gdplev.xls", skiprows=5)
    gdp_df.dropna(axis=0, how="all", inplace=True)
    gdp_df = gdp_df[
        [
            "Unnamed: 4",
            "GDP in billions of current dollars.1",
            "GDP in billions of chained 2009 dollars.1",
        ]
    ]
    gdp_df.rename(
        columns={
            "Unnamed: 4": "year_quarter_str",
            "GDP in billions of current dollars.1": "GDP in billions of current dollars",
            "GDP in billions of chained 2009 dollars.1": "GDP in billions of chained 2009 dollars",
        },
        inplace=True,
    )
    gdp_df["year_quarter"] = (
        gdp_df["year_quarter_str"]
        .str.extract(r"([0-9]{4})(q[1|2|3|4]{1})")
        .apply(lambda x: pd.Period(x[0] + x[1]), axis=1)
    )
    gdp_df = gdp_df[gdp_df["year_quarter"] >= pd.Period("2001Q1")].reset_index(
        drop=True
    )
    gdp_df["change"] = gdp_df["GDP in billions of current dollars"].diff().fillna(0)
    start = (
        (gdp_df["change"] >= 0)
        & (gdp_df["change"].shift(-1) < 0)
        & (gdp_df["change"].shift(-2) < 0)
    )
    return gdp_df.loc[
        start & (gdp_df["year_quarter"] == gdp_df.loc[start, "year_quarter"].min()),
        "year_quarter_str",
    ].item()

<IPython.core.display.Javascript object>

In [7]:
get_recession_start()

'2008q3'

<IPython.core.display.Javascript object>

In [8]:
def get_recession_end():
    """Returns the year and quarter of the recession end time as a
    string value in a format such as 2005q3"""
    gdp_df = pd.read_excel("gdplev.xls", skiprows=5)
    gdp_df.dropna(axis=0, how="all", inplace=True)
    gdp_df = gdp_df[
        [
            "Unnamed: 4",
            "GDP in billions of current dollars.1",
            "GDP in billions of chained 2009 dollars.1",
        ]
    ]
    gdp_df.rename(
        columns={
            "Unnamed: 4": "year_quarter_str",
            "GDP in billions of current dollars.1": "GDP in billions of current dollars",
            "GDP in billions of chained 2009 dollars.1": "GDP in billions of chained 2009 dollars",
        },
        inplace=True,
    )
    gdp_df["year_quarter"] = (
        gdp_df["year_quarter_str"]
        .str.extract(r"([0-9]{4})(q[1|2|3|4]{1})")
        .apply(lambda x: pd.Period(x[0] + x[1]), axis=1)
    )
    gdp_df = gdp_df[gdp_df["year_quarter"] >= pd.Period("2001Q1")].reset_index(
        drop=True
    )
    gdp_df["change"] = gdp_df["GDP in billions of current dollars"].diff().fillna(0)
    start = (
        (gdp_df["change"] >= 0)
        & (gdp_df["change"].shift(-1) < 0)
        & (gdp_df["change"].shift(-2) < 0)
    )
    end = (gdp_df["change"] > 0) & (gdp_df["change"].shift(1) > 0)
    return (
        gdp_df.loc[
            end & (gdp_df["year_quarter"] > gdp_df.loc[start, "year_quarter"].max())
        ]
        .head(1)
        .year_quarter_str.item()
    )

<IPython.core.display.Javascript object>

In [9]:
get_recession_end()

'2009q4'

<IPython.core.display.Javascript object>

In [10]:
def get_recession_bottom():
    """Returns the year and quarter of the recession bottom time as a
    string value in a format such as 2005q3"""
    start_quarter_year = get_recession_start()
    end_quarter_year = get_recession_end()
    gdp_df = pd.read_excel("gdplev.xls", skiprows=5)
    gdp_df.dropna(axis=0, how="all", inplace=True)
    gdp_df = gdp_df[
        [
            "Unnamed: 4",
            "GDP in billions of current dollars.1",
            "GDP in billions of chained 2009 dollars.1",
        ]
    ]
    gdp_df.rename(
        columns={
            "Unnamed: 4": "year_quarter_str",
            "GDP in billions of current dollars.1": "GDP in billions of current dollars",
            "GDP in billions of chained 2009 dollars.1": "GDP in billions of chained 2009 dollars",
        },
        inplace=True,
    )
    gdp_df["year_quarter"] = (
        gdp_df["year_quarter_str"]
        .str.extract(r"([0-9]{4})(q[1|2|3|4]{1})")
        .apply(lambda x: pd.Period(x[0] + x[1]), axis=1)
    )
    gdp_df = gdp_df[gdp_df["year_quarter"] >= pd.Period("2001Q1")].reset_index(
        drop=True
    )
    mid_recession = gdp_df.loc[
        (
            (gdp_df["year_quarter"] > pd.Period(start_quarter_year))
            & (gdp_df["year_quarter"] < pd.Period(end_quarter_year))
        )
    ]
    return mid_recession.loc[
        mid_recession["GDP in billions of current dollars"]
        == mid_recession["GDP in billions of current dollars"].min(),
        "year_quarter_str",
    ].item()

<IPython.core.display.Javascript object>

In [11]:
get_recession_bottom()

'2009q2'

<IPython.core.display.Javascript object>

In [12]:
def convert_housing_data_to_quarters():
    """Converts the housing data to quarters and returns it as mean
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    """
    housing_df = pd.read_csv("City_Zhvi_AllHomes.csv")
    year_cols = [
        col
        for col in housing_df.columns
        if re.match(r"[1|2]{1}[0-9]{3}\-[0|1]{1}[0-9]{1}", col)
    ]
    housing_df = housing_df[["State", "RegionName", *year_cols]]
    housing_df["State"] = housing_df["State"].map(states)
    year_quarter_map = {
        x: (
            str(pd.Period(x, freq="Q")).lower()
            if pd.Period(x, freq="Q") >= pd.Period("2000q1")
            else -1
        )
        for x in year_cols
    }
    housing_df = housing_df.rename(columns=year_quarter_map)
    housing_df.drop(columns=[-1], inplace=True)
    housing_df.set_index(["State", "RegionName"], inplace=True)

    return housing_df.groupby(housing_df.columns, axis=1).mean()

<IPython.core.display.Javascript object>

In [13]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


<IPython.core.display.Javascript object>

In [14]:
def run_ttest():
    """First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values,
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss)."""

    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    university_towns = get_list_of_university_towns()
    housing_prices_df = convert_housing_data_to_quarters()[
        [recession_start, recession_bottom]
    ]
    university_towns["is_university_town"] = 1
    housing_university_df = pd.merge(
        housing_prices_df,
        university_towns,
        left_index=True,
        right_on=["State", "RegionName"],
        how="left",
    )
    housing_university_df = housing_university_df.dropna(
        subset=[recession_start, recession_bottom]
    )
    housing_university_df.reset_index(drop=True, inplace=True)
    housing_university_df.fillna(0, inplace=True)
    housing_university_df["change"] = (
        housing_university_df[recession_bottom] - housing_university_df[recession_start]
    )

    housing_uni_town_df = housing_university_df[
        housing_university_df.is_university_town == 1
    ]
    housing_non_uni_town_df = housing_university_df[
        housing_university_df.is_university_town == 0
    ]

    p_value = ttest_ind(
        housing_uni_town_df["change"], housing_non_uni_town_df["change"]
    ).pvalue

    if p_value < 0.01:
        different = True
    else:
        different = False

    if housing_uni_town_df["change"].mean() > housing_non_uni_town_df["change"].mean():
        better = "university town"
    else:
        better = "non-university town"

    return (different, p_value, better)

<IPython.core.display.Javascript object>

In [15]:
run_ttest()

(True, 0.004325214853511201, 'university town')

<IPython.core.display.Javascript object>